In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
import os

import numpy as np
import pandas as pd
import atecml.data
from tqdm import tqdm

class BasicModel(object):
    """Parent class of basic models"""
    def train(self, x_train, y_train, x_val, y_val):
        """return a trained model and eval metric o validation data"""
        pass
    
    def predict(self, model, x_test):
        """return the predicted result"""
        pass
    
    def get_oof(self, x_train, y_train, x_test, n_folds = 5):
        num_train, num_test = x_train.shape[0], x_test.shape[0]
        oof_train = np.zeros((num_train,)) 
        oof_test = np.zeros((num_test,))
        oof_test_all_fold = np.zeros((num_test, n_folds))
        aucs = []
        model_list = []
        
        for i in range(0,n_folds):
            
            val_index = DateFold[5] #始终用最后20%验证            
            train_index = list(all_list - DateFold[i])
                            
            print('{0} fold, train {1}, val {2}'.format(i, len(train_index), len(val_index)))
            x_tra, y_tra = x_train[train_index], y_train[train_index]
            x_val, y_val = x_train[val_index], y_train[val_index]
            
            #Over_sample
            #X_resampled, y_resampled = SMOTE().fit_sample(x_tra,y_tra)
            #model, auc = self.train(X_resampled, y_resampled, x_val, y_val)
            model, auc = self.train(x_tra, y_tra, x_val, y_val)
            aucs.append(auc)
            model_list.append(model)
            oof_train[val_index] = self.predict(model, x_val)
            oof_test_all_fold[:, i] = self.predict(model, x_test)
        oof_test = np.mean(oof_test_all_fold, axis=1)
        print('all aucs {0}, average {1}'.format(aucs, np.mean(aucs)))
        return oof_train, oof_test,model_list

import lightgbm as lgb
class LGBClassifier(BasicModel):
    '''
    ' 调参范围
    'num_leaves':range(35,65,5)
    'learning_rate':[0.01,0.05,0.1,0.3,0.5,0.7]
    'min_child_weight':range(1,6,2)
    'max_depth':range(3,10,2),
    'subsample':[i/10.0 for i in range(6,10)],正常直接设置为1
    'colsample_bytree':[i/10.0 for i in range(6,10)]，正常直接设置为1
    'reg_alpha','reg_lambda':[1e-5, 1e-2, 0.1, 1, 2,2.5,3]
    '''
    def __init__(self,boost_type,boost_round=1000,early_stop=100,pos_weight=1):
        self.num_boost_round = boost_round
        self.early_stopping_rounds = early_stop
        self.params = {
            'task': 'train',
            'boosting_type': boost_type,
            'max_depth': 3,
            'metric': {'auc'},
            'nthread': 40,
            'num_leaves': 10,
            'objective': 'binary',
            'verbose' : -1,
            #'device': 'gpu'
            }
        print(self.params)
        
    def train(self, x_train, y_train, x_val, y_val):
        print('train with lgb model')
        lgbtrain = lgb.Dataset(x_train, y_train)
        lgbval = lgb.Dataset(x_val, y_val)
        model = lgb.train(self.params, 
                          lgbtrain,
                          valid_sets=lgbval, 
                          verbose_eval = 50,
                          num_boost_round = self.num_boost_round,
                          early_stopping_rounds = self.early_stopping_rounds)
        return model, model.best_score['valid_0']['auc']
    
    def predict(self, model, x_test):
        print('test with lgb model')
        return model.predict(x_test, num_iteration=model.best_iteration)

In [2]:
#训练集为第一步build的纬度提升矩阵，并过滤掉unknown标签
train_df = pd.read_pickle('./01_train.dat')
val_df =  pd.read_pickle('./01_test.dat')

#train_df = atecml.data.load_train()
#val_df = atecml.data.load_test()
train_df.loc[train_df.label == 0, 'Fraud'] = 0
train_df.loc[train_df.label != 0, 'Fraud'] = 1


In [3]:
import joblib
predictors =joblib.load('./woe_feature.dat')
#predictors = [x for x in train_df.columns if x not in atecml.data.NOT_FEATURE_SUM]


target = 'Fraud'
DateFold={}

DateFold[0] = set(atecml.data.filter_date(train_df,start_date='2017-09-05',end_date='2017-09-12').index)
DateFold[1] = set(atecml.data.filter_date(train_df,start_date='2017-09-13',end_date='2017-09-20').index)
DateFold[2] = set(atecml.data.filter_date(train_df,start_date='2017-09-21',end_date='2017-09-28').index)
DateFold[3] = set(atecml.data.filter_date(train_df,start_date='2017-09-29',end_date='2017-10-06').index)
DateFold[4] = set(atecml.data.filter_date(train_df,start_date='2017-10-07',end_date='2017-10-14').index)
DateFold[5] = list(atecml.data.filter_date(train_df,start_date='2017-10-15',end_date='2017-11-24').index)

all_list = set(train_df.index) - set(DateFold[5])
len(all_list),len(DateFold[5])

(634284, 360447)

In [4]:
x_train = np.array(train_df[predictors])
y_train = np.array(train_df[target])
x_test = np.array(val_df[predictors])
print(x_train.shape, y_train.shape, x_test.shape)

(994731, 168) (994731,) (500538, 168)


In [5]:
num_pos = np.sum(train_df[target])  
num_neg = x_train.shape[0]- num_pos
scale_pos_weight =  num_neg/num_pos
print(num_pos,num_neg,scale_pos_weight)

16847.0 977884.0 58.04499317385885


In [6]:
num_boost_round = 2000
num_early_stop = 100

In [7]:

dart_classifier = LGBClassifier(boost_type='dart',boost_round=num_boost_round,early_stop=num_early_stop,pos_weight= scale_pos_weight)
dart_oof_train, dart_oof_test,dart_model_list = dart_classifier.get_oof(x_train, y_train, x_test)
print(dart_oof_train.shape, dart_oof_test.shape)   


gbdt_classifier = LGBClassifier(boost_type='gbdt',boost_round=num_boost_round,early_stop=num_early_stop,pos_weight= scale_pos_weight)
gbdt_oof_train, gbdt_oof_test,gbdt_model_list = gbdt_classifier.get_oof(x_train, y_train, x_test)
print(gbdt_oof_train.shape, gbdt_oof_test.shape)  
 


{'min_child_samples': 20, 'metric': {'auc'}, 'num_leaves': 80, 'subsample': 0.85, 'task': 'train', 'nthread': 40, 'is_unbalance': True, 'learning_rate': 0.05, 'objective': 'binary', 'subsample_freq': 1, 'subsample_for_bin': 2000, 'boosting_type': 'dart', 'verbose': -1, 'use_missing': 'true', 'min_split_gain': 0, 'max_bin': 511, 'colsample_bytree': 0.7, 'max_depth': -1}
0 fold, train 500685, val 360447
train with lgb model
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's auc: 0.967054
[100]	valid_0's auc: 0.97075
[150]	valid_0's auc: 0.973024
[200]	valid_0's auc: 0.974198
[250]	valid_0's auc: 0.975055
[300]	valid_0's auc: 0.975691
[350]	valid_0's auc: 0.976046
[400]	valid_0's auc: 0.976443
[450]	valid_0's auc: 0.97675
[500]	valid_0's auc: 0.976812
[550]	valid_0's auc: 0.976832
[600]	valid_0's auc: 0.977054
[650]	valid_0's auc: 0.977077
[700]	valid_0's auc: 0.977027
[750]	valid_0's auc: 0.976731
Early stopping, best iteration is:
[690]	valid_0's auc: 0.977111

KeyboardInterrupt: 

In [ ]:
train_dart_r = gbdt_oof_train
n = int(gbdt_oof_train.shape[0] * 0.8)
y_pred, y_val = train_dart_r[n:], y_train[n:]
_f1,_f2,_f3 = atecml.data.accuracy_validation(y_val,y_pred)


In [ ]:
train_dart_r = dart_oof_train
n = int(dart_oof_train.shape[0] * 0.8)
y_pred, y_val = train_dart_r[n:], y_train[n:]
_f1,_f2,_f3 = atecml.data.accuracy_validation(y_val,y_pred)